In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [67]:
data = pd.read_csv('../data/mini.csv', parse_dates=['occurence'])
data.head()

,customer_id,occurence,cost,item_id
0,416705,2017-05-07 21:58:10,299.0,515274
1,13891,2018-02-10 17:35:11,1090.0,828115
2,9081,2017-12-21 17:13:44,499.0,695501
3,470904,2017-10-31 10:39:49,290.0,899821
4,58500,2018-03-09 20:57:29,150.0,518554


In [68]:
print(data['occurence'].min(), data['occurence'].max())

2017-01-01 12:33:40 2018-07-23 10:38:37


In [69]:
sd = dt.datetime(2018,8,1)
data['recency_in_days']= sd - data['occurence']
data['recency_in_days'] = data['recency_in_days'].astype('timedelta64[D]')
data.head()

,customer_id,occurence,cost,item_id,recency_in_days
0,416705,2017-05-07 21:58:10,299.0,515274,450.0
1,13891,2018-02-10 17:35:11,1090.0,828115,171.0
2,9081,2017-12-21 17:13:44,499.0,695501,222.0
3,470904,2017-10-31 10:39:49,290.0,899821,273.0
4,58500,2018-03-09 20:57:29,150.0,518554,144.0


In [70]:
data = data[data['recency_in_days'] < 730]

In [71]:
rfm_table_users = data.groupby('customer_id').agg({'recency_in_days': lambda x: x.min(), # Recency
                                            'customer_id': lambda x: len(x), # Frequency
                                            'cost': lambda x: x.sum()}) # Monetary_value
rfm_table_users.rename(columns={'recency_in_days': 'recency',
                         'customer_id': 'frequency',
                         'cost': 'monetary_value'}, inplace=True)

In [72]:
rfm_table_items = data.groupby('item_id').agg({'recency_in_days': lambda x: x.min(), # Recency
                                            'item_id': lambda x: len(x), # Frequency
                                            'cost': lambda x: x.sum()}) # Monetary_value
rfm_table_items.rename(columns={'recency_in_days': 'recency',
                         'item_id': 'frequency',
                         'cost': 'monetary_value'}, inplace=True)

In [73]:
rfm_table_users = rfm_table_users[(rfm_table_users['frequency'] > 6) & (rfm_table_users['frequency'] < 52)]
rfm_table_items = rfm_table_items[(rfm_table_items['frequency'] > 6) & (rfm_table_items['frequency'] < 105)]

In [74]:
rfm_table_items.quantile(.05)

recency              9.0
frequency            7.0
monetary_value    3600.0
Name: 0.05, dtype: float64

In [75]:
rfm_table_users.head(5)

,recency,frequency,monetary_value
customer_id,,,
1,15.0,38,66918.0
54,43.0,20,86715.0
153,219.0,11,26795.0
156,14.0,23,44317.0
158,156.0,13,88911.0


In [76]:
rfm_table_items.head(5)

,recency,frequency,monetary_value
item_id,,,
000061,178.0,10,8903.0
000064,29.0,9,8655.0
000160,58.0,15,17311.0
000509,26.0,31,43630.0
003102,10.0,51,46459.0


In [77]:
quartiles_users = rfm_table_users.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()
quartiles_items = rfm_table_items.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()
print(quartiles_users, quartiles_items, sep='\n')

{'recency': {0.2: 23.0, 0.4: 46.0, 0.6: 85.0, 0.8: 158.0}, 'frequency': {0.2: 8.0, 0.4: 11.0, 0.6: 15.0, 0.8: 23.0}, 'monetary_value': {0.2: 15402.0, 0.4: 27132.0, 0.6: 43915.0, 0.8: 76100.0}}
{'recency': {0.2: 15.0, 0.4: 29.0, 0.6: 66.0, 0.8: 176.0}, 'frequency': {0.2: 9.0, 0.4: 12.0, 0.6: 19.0, 0.8: 33.0}, 'monetary_value': {0.2: 9770.0, 0.4: 20420.0, 0.6: 39411.0, 0.8: 90445.0}}


In [78]:
def RClass(x,p,q):
    if x <= q[p][0.2]:
        return 5
    elif x <= q[p][0.4]:
        return 4
    elif x <= q[p][0.6]:
        return 3
    elif x <= q[p][0.8]:
        return 2
    else:
        return 1
    
def FMClass(x,p,q):
    if x <= q[p][0.2]:
        return 1
    elif x <= q[p][0.4]:
        return 2
    elif x <= q[p][0.6]:
        return 3
    elif x <= q[p][0.8]:
        return 4
    else:
        return 5

In [79]:
rfm_seg_users = rfm_table_users
rfm_seg_users['R_Quartile'] = rfm_seg_users['recency'].apply(RClass, args=('recency', quartiles_users))
rfm_seg_users['F_Quartile'] = rfm_seg_users['frequency'].apply(FMClass, args=('frequency',quartiles_users))
rfm_seg_users['M_Quartile'] = rfm_seg_users['monetary_value'].apply(FMClass, args=('monetary_value',quartiles_users))

In [80]:
rfm_seg_items = rfm_table_items
rfm_seg_items['R_Quartile'] = rfm_seg_items['recency'].apply(RClass, args=('recency', quartiles_items))
rfm_seg_items['F_Quartile'] = rfm_seg_items['frequency'].apply(FMClass, args=('frequency',quartiles_items))
rfm_seg_items['M_Quartile'] = rfm_seg_items['monetary_value'].apply(FMClass, args=('monetary_value',quartiles_items))

In [81]:
rfm_seg_users['RFMClass'] = rfm_seg_users.R_Quartile.map(str) \
                            + rfm_seg_users.F_Quartile.map(str) \
                            + rfm_seg_users.M_Quartile.map(str)

In [82]:
rfm_seg_items['RFMClass'] = rfm_seg_items.R_Quartile.map(str) \
                            + rfm_seg_items.F_Quartile.map(str) \
                            + rfm_seg_items.M_Quartile.map(str)

In [83]:
rfm_seg_users.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer_id,,,,,,,
1,15.0,38,66918.0,5,5,4,554
54,43.0,20,86715.0,4,4,5,445
153,219.0,11,26795.0,1,2,2,122
156,14.0,23,44317.0,5,4,4,544
158,156.0,13,88911.0,2,3,5,235


In [84]:
rfm_seg_items.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
item_id,,,,,,,
000061,178.0,10,8903.0,1,2,1,121
000064,29.0,9,8655.0,4,1,1,411
000160,58.0,15,17311.0,3,3,2,332
000509,26.0,31,43630.0,4,4,4,444
003102,10.0,51,46459.0,5,5,4,554


In [89]:
rfm_seg_users['Rating'] = rfm_seg_users['R_Quartile'] + rfm_seg_users['F_Quartile'] +rfm_seg_users['M_Quartile']
rfm_seg_items['Rating'] = rfm_seg_items['R_Quartile'] + rfm_seg_items['F_Quartile'] +rfm_seg_items['M_Quartile']
rfm_seg_users.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Rating
customer_id,,,,,,,,
1,15.0,38,66918.0,5,5,4,554,14
54,43.0,20,86715.0,4,4,5,445,13
153,219.0,11,26795.0,1,2,2,122,5
156,14.0,23,44317.0,5,4,4,544,13
158,156.0,13,88911.0,2,3,5,235,10


In [90]:
quartiles_users = rfm_table_users.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()
quartiles_items = rfm_table_items.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()
rfm_seg_users['Rating'] = rfm_seg_users['Rating'].apply(FMClass, args=('Rating', quartiles_users))
rfm_seg_items['Rating'] = rfm_seg_items['Rating'].apply(FMClass, args=('Rating', quartiles_items))

In [91]:
rfm_seg_users.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Rating
customer_id,,,,,,,,
1,15.0,38,66918.0,5,5,4,554,5
54,43.0,20,86715.0,4,4,5,445,5
153,219.0,11,26795.0,1,2,2,122,1
156,14.0,23,44317.0,5,4,4,544,5
158,156.0,13,88911.0,2,3,5,235,3


In [92]:
rfm_seg_items.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Rating
item_id,,,,,,,,
000061,178.0,10,8903.0,1,2,1,121,1
000064,29.0,9,8655.0,4,1,1,411,1
000160,58.0,15,17311.0,3,3,2,332,2
000509,26.0,31,43630.0,4,4,4,444,4
003102,10.0,51,46459.0,5,5,4,554,5


In [93]:
rfm_seg_items.sort_values(by=['RFMClass', 'monetary_value'], ascending=[False, False]).head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Rating
item_id,,,,,,,,
987548,13.0,42,7585100.0,5,5,5,555,5
1028542,12.0,43,6046112.0,5,5,5,555,5
892618,15.0,86,3416917.0,5,5,5,555,5
360511,11.0,89,3102412.0,5,5,5,555,5
766745,11.0,36,2804832.0,5,5,5,555,5
856510,15.0,63,2633036.0,5,5,5,555,5
797896,10.0,93,2584153.0,5,5,5,555,5
948390,9.0,104,2509289.0,5,5,5,555,5
712896,12.0,45,2494158.0,5,5,5,555,5


In [94]:
rfm_seg_users.sort_values(by=['RFMClass', 'monetary_value'], ascending=[False, False]).head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Rating
customer_id,,,,,,,,
60481,12.0,32,822311.00,5,5,5,555,5
476605,9.0,27,755298.00,5,5,5,555,5
425362,14.0,27,739261.76,5,5,5,555,5
494169,13.0,40,586988.00,5,5,5,555,5
432863,22.0,34,575341.00,5,5,5,555,5
296860,13.0,40,568664.00,5,5,5,555,5
576661,9.0,51,556757.25,5,5,5,555,5
11375,19.0,50,545870.00,5,5,5,555,5
117955,22.0,33,530572.00,5,5,5,555,5


In [95]:
rfm_seg_users.groupby('Rating').agg('monetary_value').mean()

Rating
1     16978.091668
2     30419.314777
3     46831.131407
4     73061.444343
5    117402.016882
Name: monetary_value, dtype: float64

In [96]:
rfm_seg_items.groupby('Rating').agg('monetary_value').mean()

Rating
1     13362.662746
2     36340.969823
3     64915.825015
4    115111.365072
5    220397.070809
Name: monetary_value, dtype: float64